In [2]:
import numpy as np
import numpy.matlib
import math
import timeit

In [3]:
d=np.loadtxt('originaldata.txt')
train=d[0:140,:] #split data
c1=np.zeros((1,8))
c2=np.zeros((1,8))
for i in range(140): #把两类螃蟹分别放到两个矩阵
    if train[i,0]==0:
        c1=np.row_stack((c1,train[i,:]))
    else:
        c2=np.row_stack((c2,train[i,:]))
c1=c1[1:,1:7] #矩阵第一行为0;性别只需要用一列，这列是1，另一列一定是0;第一行是类别，分好类后就不需要了
c2=c2[1:,1:7]
t1=train[0:140,0] #get the classes of data
t1=np.mat(t1)
t1=t1.T
t2=train[0:140,1:7] #不包括label的原所有数据，性别只用一列

[[  0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.   20.6  14.4  42.8  46.5  19.6   1.    0. ]
 [  0.   16.7  14.3  32.3  37.   14.7   0.    1. ]
 [  0.   15.6  14.1  31.   34.5  13.8   0.    1. ]
 [  0.   14.1  10.5  29.1  31.6  13.1   1.    0. ]
 [  0.   19.9  16.6  39.4  43.9  17.9   0.    1. ]
 [  0.   17.5  14.7  33.3  37.6  14.6   0.    1. ]
 [  0.   20.1  17.2  39.8  44.1  18.6   0.    1. ]
 [  0.   19.9  17.9  40.1  46.4  17.9   0.    1. ]
 [  0.   19.7  16.7  39.9  43.6  18.2   0.    1. ]
 [  0.   14.   11.5  29.2  32.2  13.1   1.    0. ]
 [  0.   17.4  12.8  36.1  39.5  16.2   1.    0. ]
 [  0.   10.2   8.2  20.2  22.2   9.    1.    0. ]
 [  0.   18.8  13.8  39.2  43.3  17.9   1.    0. ]
 [  0.   17.6  14.   34.   38.6  15.5   0.    1. ]
 [  0.   15.4  11.1  30.2  33.6  13.5   1.    0. ]
 [  0.   18.   14.9  34.7  39.5  15.7   0.    1. ]
 [  0.   18.9  16.7  36.3  41.7  15.3   0.    1. ]
 [  0.   13.7  11.   27.5  30.5  12.2   1.    0. ]
 [  0.   19.4  14.4  39.8  44.3

In [6]:
m1=np.mean(c1,axis=0) #get mean of data
m2=np.mean(c2,axis=0)
m1=np.mat(m1)
m2=np.mat(m2) #行矩阵
m11=np.zeros((1,6))
m22=np.zeros((1,6))
for i in range(len(c1)):
    m11=np.row_stack((m11,m1))
for i in range(len(c2)):
    m22=np.row_stack((m22,m2))
m11=m11[1:,0:6] #生成mean矩阵
m22=m22[1:,0:6]
v1=(((c1-m11).T)@(c1-m11))/len(c1) #get the covariance matrix，保证相乘的行和列内的数据都是属于同一类的
v2=((c2-m22).T)@(c2-m22)/len(c2) 
p1=len(c1)/140 #计算training data中每一类占的比例
p2=len(c2)/140

[[ 17.10555556  13.62083333  34.1375      38.10277778  15.46527778
    0.51388889]]


In [9]:
#其实没有所谓的traing和testing过程，只不过是把所有数据，分成两部分看效果，两部分都是平等的，
start = timeit.default_timer()
g=np.zeros((len(t2),2))  #len(t2)=140
for i in range(len(t2)): #comptute g
    #g[:,0]实际上是储存每来6个螃蟹的数据，这个螃蟹属于标签为0的类的后验概率，同理g[:,1]是属于标签为1的类的后验概率
    #下面的公式推导过程略
    g[i,0]=-0.5*(t2[i,:]-m1[0,:])@(np.linalg.inv(v1)@((t2[i,:]-m1[0,:]).T))-0.5*len(t2[0])*math.log(2*math.pi)-0.5*np.linalg.det(v1)+math.log(p1)
    g[i,1]=-0.5*(t2[i,:]-m2[0,:])@(np.linalg.inv(v2)@((t2[i,:]-m2[0,:]).T))-0.5*len(t2[0])*math.log(2*math.pi)-0.5*np.linalg.det(v2)+math.log(p2)
print(g)
r=g[:,0]-g[:,1]
r=np.mat(r)
r=r.T
for i in range(len(r)): #classify data，如果0类的概率大于1类的，那就归到0类，反之归到1类
    if r[i,0]>=0:
        r[i,0]=0
    else:
        r[i,0]=1
stop = timeit.default_timer()
n1=0
n2=0
n3=0
n4=0
for i in range(len(g)): #get the confusion matrix
    if r[i,0]==t1[i,0] and r[i,0]==0:
        n1=n1+1
    elif r[i,0]==t1[i,0] and r[i,0]==1:
        n2=n2+1
    elif r[i,0]==0:
        n3=n3+1
    elif r[i,0]==1:
        n4=n4+1
print('Training data set')
print('         Classify 0   Classify 1')
print('Class 0   ',n1,'          ',n3)
print('Class 1   ',n4,'          ',n2)
print(stop-start)

[[-10.52265109 -65.17543249]
 [-22.12299775  -9.18779586]
 [ -7.58948364 -30.20368355]
 [-18.74657358  -9.64776247]
 [ -8.64272699 -33.08748032]
 [-18.94094214  -9.39736949]
 [ -7.97438421 -33.09032143]
 [-17.13788422  -9.57468837]
 [-22.68556183  -8.39416522]
 [ -7.49872274 -53.66445689]
 [ -8.69424644 -45.76609123]
 [-10.5314107  -66.99330746]
 [-12.74229517 -30.63408715]
 [-55.55843033 -12.18286284]
 [-32.90715     -9.24292589]
 [-11.55796588 -64.75573595]
 [-16.90593982  -8.00225262]
 [ -8.76092417 -24.42264171]
 [ -7.93722686 -37.99113859]
 [ -9.11733467 -21.78101144]
 [-39.07569403 -11.35206536]
 [-21.55360669 -10.1449363 ]
 [ -8.26384943 -37.83169142]
 [ -8.55168631 -36.08962149]
 [ -7.74053576 -32.13606128]
 [-18.46488022  -7.95984558]
 [ -7.48007689 -38.38832441]
 [-30.19257596  -8.7021264 ]
 [-24.69239738 -10.15682351]
 [-15.89712351 -52.46097398]
 [-24.79282833  -9.03501132]
 [ -8.16542293 -24.61155764]
 [ -8.0678767  -34.95488504]
 [-58.97439657 -14.31993743]
 [-30.43071186

In [170]:
print(np.eye(2))
print(s1+a*np.eye(n_fea))

[[ 1.  0.]
 [ 0.  1.]]


NameError: name 'n_fea' is not defined

In [171]:
traindata=t[0:140,1:7]
testdata=t[:,0:6]

NameError: name 't' is not defined